# 自定义层

深度学习的一个魅力在于神经网络中各式各样的层，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然`torch.nn`提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何使用`torch`来自定义一个网络层，从而可以被重复调用。


## 不含模型参数的自定义层

我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和[“模型构造”](model-construction.ipynb)一节中介绍的使用`Module`类构造模型类似。下面的`CenteredLayer`类通过继承`Module`类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了`forward`函数里。这个层里不含模型参数。

In [1]:
import torch
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    
    def forward(self, x):
        return x - x.mean()

我们可以实例化这个层，然后做前向计算。

In [2]:
layer = CenteredLayer()
layer(torch.Tensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

我们也可以用它来构造更复杂的模型。

In [3]:
net = nn.Sequential(
    nn.Linear(8, 128),
    CenteredLayer()
)

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [4]:
y = net(torch.rand(4, 8))
y.mean().item()

7.450580596923828e-09

## 含模型参数的自定义层

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

对于模型的可学习参数，需要使用`slef.param_name = nn.Parameter(初始化形式)`进行创建。当Paramenter赋值给Module的属性的时候，他会自动的被加到 Module的 参数列表中(即：会出现在  parameters() 迭代器中)，并且参数名为`param_name`。

现在我们尝试实现一个含权重参数和偏差参数的全连接层。它使用ReLU函数作为激活函数。其中`in_units`和`units`分别代表输入个数和输出个数。

In [5]:
class MyLinear(nn.Module):
    # in_features为该层的输入特征数, out_features为该层的输出特征数
    def __init__(self, in_features, out_features, **kwargs):
        super(MyLinear, self).__init__(**kwargs)
        self.weight = nn.Parameter(torch.rand(in_features, out_features))
        self.bias = nn.Parameter(torch.rand(out_features))
    
    def forward(self, x):
        linear = x.mm(self.weight) + self.bias
        return torch.relu(linear)

下面，我们实例化`MyDense`类并访问它的模型参数。

In [6]:
linear = MyLinear(in_features=5, out_features=3)
for name, param in linear.named_parameters():
    print(name, param.shape, param.dtype)

weight torch.Size([5, 3]) torch.float32
bias torch.Size([3]) torch.float32


我们可以直接使用自定义层做前向计算。

In [7]:
linear(torch.rand(2, 5))

tensor([[0.5579, 1.2363, 1.2546],
        [0.7231, 1.3465, 1.1097]], grad_fn=<ReluBackward0>)

我们也可以使用自定义层构造模型。它和PyTorch的其他层在使用上很类似。

In [8]:
net = nn.Sequential(
    MyLinear(64, 8),
    MyLinear(8, 1)
)

net(torch.rand(2, 64))

tensor([[39.8864],
        [39.3420]], grad_fn=<ReluBackward0>)

## 小结

* 可以通过`Module`类自定义神经网络中的层，从而可以被重复调用。


## 练习

* 自定义一个层，使用它做一次前向计算。




## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1256)

![](../img/qr_custom-layer.svg)